# Tutorial: The Neocortical Microcircuit Collaboration Portal

Copyright © 2005-2021 Blue Brain Project/EPFL. This notebook is under a [LGPLv3](https://raw.githubusercontent.com/BlueBrain/MOOC-neurons-and-synapses-2017/master/LICENSE.txt) license.

The Neocortical Microcircuit Collaboration Portal (NMC Portal, at [https://bbp.epfl.ch/nmc-portal](https://bbp.epfl.ch/nmc-portal)) provides an online public resource of the Blue Brain Project's first release of a digital reconstruction of the microcircuitry of juvenile Rat somatosensory cortex, access to experimental data sets used in the reconstruction, and the resulting single cell models.

![NMC Portal web page](https://bbp.epfl.ch/public/SimNeuroMOOC/images/NMCPortal/NMCWeb.png)

In this tutorial we will focus on working with the electrical models of single neurons one can download from the NMC portal.

Let's begin with importing useful Python packages

In [ ]:
%matplotlib inline
import matplotlib.pyplot as pyplot

import os
import zipfile # Extract zip files
import urllib # Download files from the web
import neurom # Analyse / view morphologies
import neurom.viewer

## Downloading the model

We can [select](https://bbp.epfl.ch/nmc-portal/microcircuit.html#/metype/L5_TTPC2_cADpyr/details) a model from the NMC Portal, and download the zip file containing the code of the model. Let us download a Neocortical Layer 5 Thick Tufted Pyramidal cell model:

In [ ]:
urllib.request.urlretrieve('https://bbp.epfl.ch/nmc-portal/assets/documents/static/downloads-zip/L5_TTPC2_cADpyr232_1.zip','L5_TTPC2_cADpyr232_1.zip');

Then we extract this zip file on disk:

In [ ]:
import os, zipfile
with zipfile.ZipFile('L5_TTPC2_cADpyr232_1.zip', 'r') as zip_file:
    zip_file.extractall('.')

Now we change directory to the model dir. (**Be careful**, once this line is executed don't execute the lines above because it will create nested subdirectories)

In [ ]:
os.chdir('L5_TTPC2_cADpyr232_1')

The model contains MOD files describing the NEURON simulator mechanisms used in the model. As we have seen in another tutorial, one can compile these files using the 'nrnivmodl' command:

In [ ]:
!nrnivmodl mechanisms

## Visualizing the morphology

The morphology of the cell is located in the 'morphology' subdirectory. 
We can visualize this morphology using NeuroM:

In [ ]:
neurom.viewer.draw(neurom.load_neuron('morphology/dend-C060114A7_axon-C060116A3_-_Clone_2.asc'));

## Initializing the model

To load the functionality of the electrical model we load the 'init.hoc' HOC file.

In [ ]:
%%capture
import neuron as nrn # NEURON simulator
nrn.h.load_file("./init.hoc")

The init.hoc has created a function 'create_cell' to instantiate the neuron in the NEURON simulator. As argument it takes a value to enable/disable the synapses of the cell. (1=True, 0=False)

In [ ]:
%%capture
nrn.h.create_cell(1) # argument 1 stands for 'load synapses'

After this function has been called the global name space of the NEURON simulator contains a variable 'cell'. 
Let's create a python variable to point to this object and for the soma:

In [ ]:
cell = nrn.h.cell
soma = cell.soma[0]

## Injecting a step current

Now that we have a pointer to the soma of the neuron, we can inject a current into it. As we've learned in another tutorial, we can create an IClamp to inject the current:

In [ ]:
# Mention source of amplitude
stimulus = nrn.h.IClamp(0.5, sec=soma)
stimulus.dur = 400 # ms
stimulus.delay = 100  # ms     
stimulus.amp = 0.691907 # nA

The current amplitude is exactly the value used to generate the trace for the highest step current on the [model page](https://bbp.epfl.ch/nmc-portal/microcircuit.html#/metype/L5_TTPC2_cADpyr/details).
The values to recreate the three traces can be retrieved from a file in the model package:

In [ ]:
with open('current_amps.dat') as current_file:
    current_content = current_file.read()

print('File content: ', current_content)
holding_current, step1_current, step2_current, step3_current = [float(x) for x in current_content.split()]

print('')
print('Holding current: %f nA' % holding_current)
print('Step 1: %f nA' % step1_current)
print('Step 2: %f nA' % step2_current)
print('Step 3: %f nA' % step3_current)

The 'holding current' is current the experimentalists inject during the entire recording to hold the neuron at a predefined voltage before the actual stimulus is applied.

The init.hoc also has provided us with a function to enable the recording of the voltage:

In [ ]:
nrn.h.create_recording();    #activate recording of activity 

Now we can run the simulation:

In [ ]:
nrn.h.tstop = 600 # ms
nrn.h.dt = 0.05 # 
nrn.h.run();

nrn.h.save_recording();

And plot the time/voltage:

In [ ]:
time = nrn.h.time
voltage = nrn.h.voltage

def plot_tv(time_array, voltage_array, label=None, constants=[]):
    import matplotlib.pyplot as plt
    import numpy
    plt.plot(time_array, voltage_array, label=label)
    for constant in constants:
        plt.plot(time_array, constant*numpy.ones(len(time_array)))
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane voltage (mV)')
    

plot_tv(time, voltage);

## Exercise

To reduce the simulation time, the simulation ends at 600 ms. 
Try to replicate the exact traces on the NMC portal.
The trace shown on the portal is the result of a 3000 ms simulation. The stimulus starts at 700 ms, and last for 2000 ms. Create two iclamps and inject a holding current for the full duration of the simulation, and step current for 2000 ms. Used the amplitudes retrieved from the current_amps.dat file.

## Activating synapses

To handle the synapses of the cell we will make use of 'synapses' attribute of the cell object. E.g. all the synapse objects are stored in the synapse_list attribute. We can print the number of synapses on this neuron: 

In [ ]:
print("Number of synapses: ", len(list(cell.synapses.synapse_list)))

Similarly, all the neurons that are presynaptic to this cell are represented as a netstim object. A netstim is a NEURON mechanism that acts as a virtual cell. It can connect to a synapse and e.g. inject a poisson spike train into the synapse. All the presynaptic netstims are in the netstim_list:

In [ ]:
print("Number of pre-synaptic cells: ", len(list(cell.synapses.netstim_list)))

As you can see there are less presynaptic cells than synapses. This is because cells can project to several synapses on the postsynaptic cell at the same time (these are called a multisynaptic connections).

The presynaptic connections are ordered according the morphological type (m-type) of the presynaptic cells.
In the neocortical microcircuit model there are 55 m-types defined. A glossary of all the abreviations used in the m-types is available on the [NMC portal website](https://bbp.epfl.ch/nmc-portal/glossary.html).
The model package we downloaded from the NMC portal has a file that maps every m-type to an integer:

In [ ]:
with open('synapses/mtype_map.tsv') as mtype_map_file:
    mtype_map_content = mtype_map_file.read()
print('integer\tm-type')
print('---------------')
print(mtype_map_content)

This map is very useful, because many lists in the model code are ordered according to m-type. So we will store this map in a dictionary for later use: 

In [ ]:
mtype_map = {}
for line in mtype_map_content.split('\n')[:-1]:
    n, mtype = line.split()
    mtype_map[mtype] = int(n)
    
print('Integer for L23_PC: %d' % mtype_map['L23_PC'])

Now we can e.g. enable all the synapses that are projected onto this cell from L23_PC (layer 2/3 pyramidal cell).
The active_pre_mtypes is a NEURON [vector](https://www.neuron.yale.edu/neuron/static/docs/help/neuron/general/classes/vector/vect.html) (it uses the .x syntax to access the elements). The index of the vector is determined by the m-type map. When the value in the vector is 1 all the presynaptic neurons of a particular m-types are active (and inactive when it is 0)

In [ ]:
def init_synapses(enabled_mtypes=[]):
    enabled_mtype_ints = [mtype_map[mtype] for mtype in enabled_mtypes]
    
    for i in range(0, int(cell.synapses.n_of_mtypes)): # Loop over all the m-type
        if i in enabled_mtype_ints: # Enable synapses
            cell.synapses.were_active_pre_mtypes.x[i]= 0
            cell.synapses.active_pre_mtypes.x[i] = 1        
        else: # Disable synapses
            cell.synapses.were_active_pre_mtypes.x[i]= 1
            cell.synapses.active_pre_mtypes.x[i] = 0
            
    cell.synapses.update_synapses(nrn.h.synapse_plot); # Update the synapses

init_synapses(enabled_mtypes=['L23_PC'])

When now we run a simulation we see the voltage recorded in the soma 
of the postsynaptic cell when all the L23_PC presynaptic cells are firing a poisson spike train:

In [ ]:
# Remove any previously defined stimulus
if 'stimulus' in locals():
    del stimulus

def run_plot():
    nrn.h.tstop = 600
    nrn.h.dt = 0.05

    nrn.h.run()
    nrn.h.save_recording()

    plot_tv(time, voltage)
    
run_plot()

As you can see the cell is not very active. Let's activate all the Layer 5 Thick Tufted Pyramidal cell connections instead:

In [ ]:
init_synapses(enabled_mtypes=['L5_TTPC2'])
run_plot()

We can also counteract the spiking by e.g. activating some inhibitory interneurons together with the excitatory input

In [ ]:
init_synapses(enabled_mtypes=['L23_PC', 'L23_MC', 'L23_BTC', 'L23_LBC', 'L23_SBC', 'L23_DBC'])
run_plot()

One can also influence parameters of e.g. the poisson processes that are creating the spike timings of the presynaptic cells. This is done by manipulating the attributes of the [Netstim](https://www.neuron.yale.edu/neuron/static/docs/help/neuron/neuron/mech.html#NetStim) objects.

In [ ]:
init_synapses(enabled_mtypes=['L5_TTPC2'])

pre_mtype_netstims = list(cell.synapses.pre_mtype_netstimlists)

for netstims in pre_mtype_netstims:
    for netstim in netstims: 
        netstim.interval = 10000 # ms, Average interval between spikes in the Poisson process   

run_plot()